# Eksempel på bruk av API med Dapla Statbank-klient

Dapla Statbank-klienten er laget av Statistisk sentralbyrå. Denne er hovedsakelig for interne brukere, men har noen funksjoner som gjør det enklere for eksterne brukere å hente data som JSON-stat2.

Se: _https://pypi.org/project/dapla-statbank-client/_

pip install dapla-statbank-client

In [ ]:
from statbank import apidata_all, apidata, apidata_rotate

_apidata_all_ trenger ikke en spesifisert spørring, den vil bygge sin egen spørring og prøve å hente alle data fra tabellen. Vær oppmerksom på at dette lett kan overskride grensen på 800.000 celler, inkl. tomme celler, og resulterer i en feil (429).

For å få tabeller på engelsk ved bruk av apidata_all må du oppgi den fullstendige URL-en. 

Vi ønsker  Makroøkonomiske hoveedstørrelser for Norge fra Konjunkturtendensene, tabell 12880.

## Bruk av apidata_all

In [ ]:
df_12880 = apidata_all("12880")

In [ ]:
df_12880

_apidata_rotate_ er en tynn wrapper rundt pandas.pivot-funksjonen

In [ ]:
df_12880_rotate = apidata_rotate(df_12880, "statistikkvariabel", "value")

In [ ]:
df_12880_rotate

I "Konjunkturtendenser" er de siste 4 årene prognoser.

Identifiser de siste 4 kolonnene.

In [ ]:
last_4_cols = df_12880_rotate.columns[-4:]

Sett skriften på prognosene til blå. Ved "styling" må jeg også eksplisitt sette antall desimaler som vises.

In [ ]:
styled_df_12880_rotate = df_12880_rotate.style.format(precision=1).set_properties(
    subset=last_4_cols,
    color='blue'
    )
    

In [ ]:
styled_df_12880_rotate

## Bruk av apidata

I _apidata_ kan du også spesifisere en spørring. I denne spørringen begrenser jeg årene til å være de siste 15 årene ved å bruke filteret "top".

In [ ]:
query = {
  "query": [
    {
      "code": "ContentsCode",
      "selection": {
        "filter": "all",
        "values": [
          "*"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "top",
        "values": [
           "15"
        ]
      }
    }
  ]
}

In [ ]:
df_12880_apidata = apidata("12880", query)

In [ ]:
df_12880_apidata

Pivot-tabell

In [ ]:
df_12880_apidata_rotate = apidata_rotate(df_12880_apidata, "statistikkvariabel", "value")

Sett prognoser til fet blå og venstrejuster tabellhodene

In [ ]:
styled_df_12880_apidata_rotate = df_12880_apidata_rotate.style.format(precision=1).set_properties(
    subset=last_4_cols,
    **{'color': 'blue', 'font-weight': 'bold'} 
).set_table_styles([
    {'selector': 'th.row_heading', 'props': [('text-align', 'left')]}
])

In [ ]:
styled_df_12880_apidata_rotate

### Table metdadata

In [ ]:
from statbank import apimetadata, apicodelist


In [ ]:
meta = apimetadata("12880")

In [ ]:
meta

In [ ]:
all_codelists = apicodelist("12880")

In [ ]:
all_codelists

In [ ]:
contentscode_codelist = apicodelist("12880", "ContentsCode")

In [ ]:
contentscode_codelist

Opsjonen _include_id=True_ returnerer både kode og tekst

In [ ]:
df_12880_apidata_id = apidata("12880", query, include_id=True)

In [ ]:
df_12880_apidata_id